In [235]:
import json, folium
import pandas as pd
import numpy as np
import re

In [236]:
df = pd.read_csv('data/경기도도시공원정보표준데이터.csv', encoding='euc-kr')

In [237]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename,encoding='utf-8'))

In [238]:
df = df[['소재지도로명주소','공원면적','위도','경도']]
df

,소재지도로명주소,공원면적,위도,경도
0,경기도 이천시 설봉로 30번길 22,2526.0,37.278561,127.438819
1,경기도 이천시 어재연로 10번길 25,3830.0,37.278010,127.444436
2,경기도 이천시 설봉로 81번길 34,1757.0,37.284902,127.437589
3,경기도 이천시 향교로 53번길 23,1650.0,37.285854,127.443215
4,경기도 이천시 설봉로 174(창전동),1300.0,37.285969,127.445970
...,...,...,...,...
494,경기도 수원시 영통구 청명로 151,137062.5,37.257187,127.072059
495,경기도 수원시 영통구 반달로 77,16015.3,37.250874,127.078858
496,경기도 수원시 영통구 매영로 366,11753.7,37.250663,127.071687
497,경기도 수원시 영통구 덕영대로 1555번길 60,13502.4,37.248678,127.061893


In [239]:
df.공원면적 = df.공원면적.astype(float).astype(int)

In [240]:
df_전체면적 = df.공원면적.sum()
df['구별면적비율'] = ((df['공원면적'] / df_전체면적) * 100).round(3)

In [241]:
df = df[df.소재지도로명주소.str.contains('시군') | df.소재지도로명주소.str.contains('구')]
df.head()

,소재지도로명주소,공원면적,위도,경도,구별면적비율
18,경기도 용인시 기흥구 신갈로 12,1650,37.267633,127.105818,0.012
19,경기도 용인시 기흥구 갈곡로12번길 1,189796,37.272744,127.129559,1.338
20,경기도 용인시 기흥구 기흥로116번길 10,81927,37.286179,127.113600,0.578
21,경기도 용인시 기흥구 중부대로55번길 59,5937,37.269301,127.075521,0.042
22,경기도 용인시 기흥구 중부대로 495,1901,37.281335,127.120333,0.013


In [242]:
df['시군구'] = df.소재지도로명주소.apply(lambda x : ''.join(x.split()[1:]))
df.head()

C:\Users\YONSAI\AppData\Local\Temp\ipykernel_3140\3065437389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시군구'] = df.소재지도로명주소.apply(lambda x : ''.join(x.split()[1:]))


,소재지도로명주소,공원면적,위도,경도,구별면적비율,시군구
18,경기도 용인시 기흥구 신갈로 12,1650,37.267633,127.105818,0.012,용인시기흥구신갈로12
19,경기도 용인시 기흥구 갈곡로12번길 1,189796,37.272744,127.129559,1.338,용인시기흥구갈곡로12번길1
20,경기도 용인시 기흥구 기흥로116번길 10,81927,37.286179,127.113600,0.578,용인시기흥구기흥로116번길10
21,경기도 용인시 기흥구 중부대로55번길 59,5937,37.269301,127.075521,0.042,용인시기흥구중부대로55번길59
22,경기도 용인시 기흥구 중부대로 495,1901,37.281335,127.120333,0.013,용인시기흥구중부대로495


In [243]:
df.set_index('시군구',inplace=True)
df.head()

,소재지도로명주소,공원면적,위도,경도,구별면적비율
시군구,,,,,
용인시기흥구신갈로12,경기도 용인시 기흥구 신갈로 12,1650,37.267633,127.105818,0.012
용인시기흥구갈곡로12번길1,경기도 용인시 기흥구 갈곡로12번길 1,189796,37.272744,127.129559,1.338
용인시기흥구기흥로116번길10,경기도 용인시 기흥구 기흥로116번길 10,81927,37.286179,127.113600,0.578
용인시기흥구중부대로55번길59,경기도 용인시 기흥구 중부대로55번길 59,5937,37.269301,127.075521,0.042
용인시기흥구중부대로495,경기도 용인시 기흥구 중부대로 495,1901,37.281335,127.120333,0.013


In [247]:
map = folium.Map(location=[37.4,127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df.구별면적비율,        #단계구분도로 보여줄 데이터
    columns=[df.index,df.구별면적비율],      # 데이터 프레임에서 추출할 column
    fill_color='YlOrRd',       # Color map
    key_on='properties.name'
).add_to(map)
map